In [1]:
sc.install_pypi_package('pandas==1.0.3')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1654156076759_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc.install_pypi_package("boto3")
sc.addPyFile("s3://crpytoconven/project/supp/data_preprocessing.py")
import glob, os
import numpy as np
import boto3
from pyspark.sql.functions import col, unix_timestamp, to_date
from pyspark.sql.types import DateType
from data_preprocessing import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# raw dataset
cryptocurrencies = spark.read.csv('s3://crpytoconven/project/data/coins_2015_2022.csv',header=True)
cryptocurrencies = cryptocurrencies[['symbol', 'open', 'high', 'low', 'close', 'time_open']]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
cryptocurrencies = cryptocurrencies.withColumnRenamed("symbol","Symbol") \
    .withColumnRenamed("open","Open") \
    .withColumnRenamed("high","High") \
    .withColumnRenamed("close","Close") \
    .withColumnRenamed("time_open","Date")

# cleaned reference
coins = spark.read.csv('s3://crpytoconven/project/data/crypto_2015_2022.csv',header=True) 
coins=coins.drop('_c0')

currs = coins.columns[1:]
cryp_dataset = {}
for curr in currs:
    sup = cryptocurrencies[cryptocurrencies.Symbol == curr]
    cryp_dataset[curr+'USD'] = sup

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# my data pipeline
myDataPip = DataPipline(start_date='2015-09-01', end_date='2022-05-01')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# fill dataset
for curr in cryp_dataset.keys():
    cryp_dataset[curr] = myDataPip.fill_dataset(dataset=cryp_dataset[curr], start_date='2015-09-01', end_date='2022-05-01' ,spark=spark, _type="crpyto")
    
# unify unit
new_cryp_dataset = {}
for curr in cryp_dataset.keys():
    dataset, name = myDataPip.unify_unit(cryp_dataset[curr], curr)
    new_cryp_dataset[name] = dataset
del cryp_dataset

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# activate preprocessing procedure
myDataPip.activate(crypto=new_cryp_dataset, spark=spark)